#### 네네치킨 {지점명(jijum), 전화번호(tel), 주소(addr), x, y} 딕셔너리-> 리스트에담기, 지도찍기
- 도메인명.tinydb로 제출
- 지도출력

In [1]:
import requests
from bs4 import BeautifulSoup
import folium

In [2]:
## 지점명, 전화번호, 주소 크롤링 함수.

def get_request(page):
    url='https://nenechicken.com/17_new/sub_shop01.asp'
    params = {'page':page,'ex_select':1,'ex_select2':'','IndexSword':'','GUBUN':'A'}
    response = requests.get(url,params=params)
    if response.status_code == 200:
        return response.text            
    else:
        print(response.status_code)

In [3]:
## 리스트에 데이터 각각 넣기

def add_list(page, addr, jijum, tel):
    response = get_request(page)
    soup = BeautifulSoup(response)
    shop_map = soup.select('div.shopMap')
    shops = soup.select('div.shop')
    for shop in shop_map:
        map_one = shop.a['href'].strip("JAVASCRIPT:codeAddress('');")
        addr.append(map_one)
    
    
    for shop in shops:
        jijum_one = shop.select_one('div.shopName').text
        jijum.append(jijum_one)
        tel_one = shop.select_one('span.tooltiptext').text.replace('-','')
        tel.append(tel_one)
    


In [4]:

addr=[]; jijum=[]; tel=[]
for page in range(1,48):
    add_list(page, addr, jijum, tel)

In [5]:
## 딕셔너리 리스트에 넣기.

NeNe_Chicken=[]
for i in range(len(addr)):
    dic = {'jijum':jijum[i],'tel':tel[i],'address':addr[i]}
    NeNe_Chicken.append(dic)

In [6]:
print(len(NeNe_Chicken))

1119


In [7]:
## 위도경도 뽑는 함수.

def LatLng(addr):
    app_key =  'KakaoAK '+'329ce7db5387b8388234ac4f823c3cb4'
    url = 'https://dapi.kakao.com/v2/local/search/address.json'
    params = {
        'query':addr,
        'page':'',
        'AddressSize':''
    }
    headers = {'Authorization':app_key}
    response = requests.get(url,headers=headers,params=params)
    status_code = response.status_code
    if status_code == 200:
        address_info = response.json()
        x = address_info['documents'][0]['x']
        y = address_info['documents'][0]['y']
        return [x,y]
    else:
        print(0,0)
    

In [8]:
## 위도(x)가 작은거 경도(y)가 큰거

LatLng(NeNe_Chicken[0]['address'])[0]  ##경도

'127.897536463576'

In [9]:
## 주소 잘못된 지점 따로 빼기

error_addr=[]
error_index=[]
for i in range(len(NeNe_Chicken)):
    try:
        addr = NeNe_Chicken[i]['address']
        x = LatLng(addr)[1]
        y = LatLng(addr)[0]
        NeNe_Chicken[i]['x'] = x
        NeNe_Chicken[i]['y'] = y
    except:
        error_index.append(i)
        error_addr.append(addr)

In [10]:
error_address={22:'강원도원주시지정면오크밸리1길70',
               91:'경기도고양시일산동구고봉로604-3',
               126:'경기도고양시덕양구권율대로831',
               243:'서울특별시서초구서초중앙로24길55',
               281:'서울특별시강서구등촌로51길9',
               329:'경기도김포시고촌읍은행영사정로35',
               345:'경기도시흥시정왕천로427번길6-1',
               421:'경기도성남시분당구중앙공원로54',
               425:'경기도광주시오포읍신현로44-10',
               495:'경기도안산시단원구선부광장서로39',
               513:'경기도화성시봉담읍오래2길5',
               550:'충청남도홍성군홍성읍문화로116',
               567:'',
               693:'경상북도구미시상사서로12길3',
               702:'경상북도경산시계양로8길38',
               727:'경상북도김천시영남대로2040-6',
               729:'경상북도김천시시청로40',
               735:'경북 구미시 고아읍 문장로22길 18',
               789:'울산중구동천4길15',
               831:'부산광역시기장군기장읍차성로436번길6-2',
               843:'부산광역시사하구서포로30번길3',
               913:'경상남도양산시물금읍화합3길6-16',
               922:'경상남도김해시진영읍장등로122번길5',
               938:'경상남도창녕군남지읍남지중앙2길69',
               955:'경상남도창원시성산구대암로116',
               960:'경상남도거제시일운면거제대로2491',
               970:'경상남도창원시의창구원이대로69번길30'
              }

In [13]:
## 에러주소 정확하게 바꾸기

for i in error_index:
    NeNe_Chicken[i]['address']=error_address[i]


In [16]:
## 바꾼주소 위도 경도값 딕셔너리에 추가.

error_index1=[]
for i in error_index:
    try:
        addr = NeNe_Chicken[i]['address']
        x = LatLng(addr)[1]
        y = LatLng(addr)[0]
        NeNe_Chicken[i]['x'] = x
        NeNe_Chicken[i]['y'] = y
    except:
        error_index1.append(i)

0 0


In [19]:
## 없어진 가게 리스트에서 삭제.

NeNe_Chicken.pop(567)
print(len(NeNe_Chicken))

1118


In [26]:
NeNeChicken = folium.Map(location=(NeNe_Chicken[100]['x'],NeNe_Chicken[100]['y']), zoom_start=12)
for i in range(len(NeNe_Chicken)):
    code = (NeNe_Chicken[i]['x'],NeNe_Chicken[i]['y'])
    folium.Marker(code, icon=folium.Icon(color='darkpurple',icon='glyphicon glyphicon-flag')).add_to(NeNeChicken)


In [27]:
NeNeChicken

In [28]:
from tinydb import TinyDB, Query

In [29]:
db = TinyDB('nenechicken.com.tinydb')

In [30]:
for item in NeNe_Chicken:
    db.insert(item)

In [31]:
db.all()

[{'jijum': '강원홍천군홍천점',
  'tel': '0334357983',
  'address': '강원도홍천군홍천읍갈마로4',
  'x': '37.6919217948951',
  'y': '127.897536463576'},
 {'jijum': '강원양구군방산점',
  'tel': '0334818892',
  'address': '강원도양구군방산면장거리길18',
  'x': '38.2084897631548',
  'y': '127.948921652683'},
 {'jijum': '강원춘천시석사점',
  'tel': '0332626007',
  'address': '강원도춘천시우석로79번길4-1',
  'x': '37.8653926852676',
  'y': '127.753613952182'},
 {'jijum': '강원춘천시퇴계점',
  'tel': '0332428119',
  'address': '강원도춘천시춘주로187번길20-4',
  'x': '37.8607404998532',
  'y': '127.726092979794'},
 {'jijum': '강원홍천군시동점',
  'tel': '0334329288',
  'address': '강원도홍천군남면시동안로321번길34-2',
  'x': '37.5938125815898',
  'y': '127.842106797155'},
 {'jijum': '강원인제군신남점',
  'tel': '0334610049',
  'address': '강원도인제군남면신남로23',
  'x': '37.9659580841716',
  'y': '128.077282888541'},
 {'jijum': '강원춘천시교동후평점',
  'tel': '0332448892',
  'address': '강원도춘천시성심로23-1',
  'x': '37.8814654454193',
  'y': '127.739980687183'},
 {'jijum': '강원화천군산양리점',
  'tel': '0334424484',
  'address': '강원